In [91]:
from minisam import *
import numpy as np
import liegroups.numpy
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation

In [90]:
def quaternion_angular_error(q1, q2):
    """
    angular error between two quaternions
    :param q1: (4, )
    :param q2: (4, )
    :return:
    """
    d = abs(np.dot(q1, q2))
    d = min(1.0, max(-1.0, d))
    theta = 2 * np.arccos(d) * 180 / np.pi
    return theta

In [125]:
def rotation_error(truth_rotation, predicted_rotation, is_print=True):
    rotation_errors = [quaternion_angular_error(q1, q2) for q1, q2 in zip(truth_rotation, predicted_rotation)]
    rotation_errors = np.array(rotation_errors)
    if is_print:
        print("Mean rotation error: {}".format(np.mean(rotation_errors)))
        print("Median rotation error: {}".format(np.median(rotation_errors)))
    return np.mean(rotation_errors), np.median(rotation_errors)

In [44]:
def make_lines(markers1, markers2):
    result = []
    for x1, x2 in zip(markers1, markers2):
        result.append(x1)
        result.append(x2)
        result.append(None)
    return result

In [45]:
def show_trajectories(shown_truth_trajectory, shown_predicted_trajectory):
    fig = go.Figure(
        data=[
            go.Scatter3d(x=shown_truth_trajectory[:, 0],
                         y=shown_truth_trajectory[:, 1], 
                         z=shown_truth_trajectory[:, 2],
                         mode="markers",
                         name="truth",
                         marker={
                             "size": 2
                         }),
            go.Scatter3d(x=shown_predicted_trajectory[:, 0],
                         y=shown_predicted_trajectory[:, 1],
                         z=shown_predicted_trajectory[:, 2],
                         mode="markers",
                         name="predicted",
                         marker={
                             "size": 2
                         }),
            go.Scatter3d(x=make_lines(shown_truth_trajectory[:, 0], shown_predicted_trajectory[:, 0]),
                         y=make_lines(shown_truth_trajectory[:, 1], shown_predicted_trajectory[:, 1]),
                         z=make_lines(shown_truth_trajectory[:, 2], shown_predicted_trajectory[:, 2]),
                         mode="lines",
                         name="errors")
        ]
    )
    fig.show()

In [46]:
def se3_position(position):
    position = position.astype(np.float64)
    u, d, v = np.linalg.svd(position[:3, :3])
    position[:3, :3] = u @ v
    return SE3(position)

# Load data

In [194]:
data = np.load("../metrics/chess_trajectories.npy", allow_pickle=True).item()
mean_matrix = data["mean_matrix"].astype(np.float64)[:1000]
truth_position = data["truth_position"].astype(np.float64)[:1000]
truth_rotation = data["truth_rotation"].astype(np.float64)[:1000]
predicted_position = data["predicted_position"].astype(np.float64)[:1000]
predicted_rotation = data["predicted_rotation"].astype(np.float64)[:1000]
inverse_sigma_matrix = data["inverse_sigma_matrix"].astype(np.float64)[:1000]

# Position graph optimization

In [195]:
graph = FactorGraph()
initials = Variables()

for i in range(len(inverse_sigma_matrix)):
    initials.add(key('x', i), se3_position(mean_matrix[i, 0]))
    for j in range(len(inverse_sigma_matrix[i])):
        priorLoss = GaussianLoss.SqrtInformation(inverse_sigma_matrix[i, j])
        graph.add(PriorFactor(key('x', i), se3_position(mean_matrix[i, j]), priorLoss))
    


In [196]:
optimizer_param = LevenbergMarquardtOptimizerParams()
optimizer = LevenbergMarquardtOptimizer(optimizer_param)

results = Variables()
status = optimizer.optimize(graph, initials, results)
if status != NonlinearOptimizationStatus.SUCCESS:
    print("optimization error: ", status)

In [197]:
optimized_matrix = [results.at(key("x", i)).matrix() for i in range(len(mean_matrix))]
optimized_matrix = np.array(optimized_matrix)
optimized_trajectory = optimized_matrix[:, :3, 3]

In [198]:
position_errors = np.linalg.norm(truth_position - optimized_trajectory, axis=1)
print("Mean position error: {}".format(np.mean(position_errors)))
print("Median position error: {}".format(np.median(position_errors)))

Mean position error: 0.5619693373264548
Median position error: 0.5467223106060313


In [199]:
position_errors = np.linalg.norm(truth_position - predicted_position, axis=1)
print("Mean position error: {}".format(np.mean(position_errors)))
print("Median position error: {}".format(np.median(position_errors)))

Mean position error: 0.5581361830726659
Median position error: 0.543152317718782


In [200]:
rotation_error(truth_rotation, Rotation.from_matrix(optimized_matrix[:, :3, :3]).as_quat())

Mean rotation error: 21.650836330215146
Median rotation error: 18.801060796970987


(21.650836330215146, 18.801060796970987)

In [201]:
rotation_error(truth_rotation, predicted_rotation)

Mean rotation error: 21.774547266281036
Median rotation error: 19.099696336522985


(21.774547266281036, 19.099696336522985)

In [202]:
means = []
medians = []
for j in range(mean_matrix.shape[1]):
    mean, median = rotation_error(truth_rotation, Rotation.from_matrix(mean_matrix[:, j, :3, :3]).as_quat(), False)
    means.append(mean)
    medians.append(median)

In [203]:
print(medians)

[18.666654302871216, 18.960786332041575, 17.969412741877825, 18.065969733208156]


# Beacon graph optimization

In [204]:
def matrix_from_quaternion_translation(rotation, translation):
    matrix = np.zeros((rotation.shape[0], 4, 4))
    matrix[:, :3, :3] = Rotation.from_quat(rotation).as_matrix()
    matrix[:, :3, 3] = translation
    matrix[:, 3, 3] = 1
    return matrix

In [205]:
truth_matrix = matrix_from_quaternion_translation(truth_rotation, truth_position)

In [206]:
graph = FactorGraph()
initials = Variables()

for j in range(mean_matrix.shape[1]):
    initials.add(key('x', j), se3_position(np.eye(4)))

for i in range(len(inverse_sigma_matrix)):
    for j in range(len(inverse_sigma_matrix[i])):
        priorLoss = GaussianLoss.SqrtInformation(inverse_sigma_matrix[i, j])
        delta = truth_matrix[i] @ np.linalg.inv(mean_matrix[i, j])
        graph.add(PriorFactor(key('x', j), se3_position(delta), priorLoss))

In [207]:
optimizer_param = LevenbergMarquardtOptimizerParams()
optimizer = LevenbergMarquardtOptimizer(optimizer_param)

results = Variables()
status = optimizer.optimize(graph, initials, results)
if status != NonlinearOptimizationStatus.SUCCESS:
    print("optimization error: ", status)

optimization error:  NonlinearOptimizationStatus.ERROR_INCREASE


In [208]:
deltas_matrix = [results.at(key("x", i)).matrix() for i in range(mean_matrix.shape[1])]
deltas_matrix = np.array(deltas_matrix)

In [209]:
Rotation.from_matrix(deltas_matrix[:, :3, :3]).as_euler('xyz').round(3)

array([[-0.106,  0.146, -0.036],
       [-0.076, -0.05 , -0.04 ],
       [ 0.043,  0.149,  0.065],
       [ 0.046,  0.007,  0.015]])

In [159]:
480. / 585 / 8

0.10256410256410256